In [56]:
import pandas as pd
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

c:\Users\calys\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [75]:
data = pd.read_csv('tweets-data/fufufafa.csv')
data.head()

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1835847615101243472,Tue Sep 17 23:55:49 +0000 2024,3,@UusRsd @msaid_didu Yuk ganyang Fufufafa sesua...,1836192364668588124,NaN,UusRsd,in,NaN,0,0,0,https://x.com/makelarcapres/status/18361923646...,1655621733548965889,makelarcapres
1,1836170303913009302,Tue Sep 17 23:55:41 +0000 2024,0,@kompascom Biasa....drama mulyono...pura2 kage...,1836192330896019567,NaN,kompascom,in,NaN,0,0,0,https://x.com/BantengDotID/status/183619233089...,1758470098623553538,BantengDotID
2,1835807033699999938,Tue Sep 17 23:55:11 +0000 2024,0,@fufu__fafa Jamet menjijikkan. Mesum pula.,1836192205847114084,NaN,fufu__fafa,in,"Jakarta Capital Region, Indone",0,0,0,https://x.com/kucingtuabangka/status/183619220...,1658730412720685056,kucingtuabangka
3,1836116467634545003,Tue Sep 17 23:55:00 +0000 2024,9,@satriohendri Nebeng temen...ato nebeng sopir ...,1836192161693602201,https://pbs.twimg.com/media/GXt3CR9akAMVaUU.jpg,satriohendri,in,NaN,0,0,0,https://x.com/BantengDotID/status/183619216169...,1758470098623553538,BantengDotID
4,1835960764395667947,Tue Sep 17 23:54:58 +0000 2024,0,@andikamalreza kasian org ini.. model penakut....,1836192150792650817,NaN,andikamalreza,in,NaN,0,0,0,https://x.com/bram24hugery/status/183619215079...,1809583978195906560,bram24hugery


In [76]:
data = data.drop(['id_str', 'conversation_id_str', 'image_url', 'tweet_url', 'user_id_str', 'username', 'in_reply_to_screen_name','quote_count',	'reply_count',	'retweet_count','favorite_count'], axis=1)


In [77]:
data.head()

,created_at,full_text,lang,location
0,Tue Sep 17 23:55:49 +0000 2024,@UusRsd @msaid_didu Yuk ganyang Fufufafa sesua...,in,NaN
1,Tue Sep 17 23:55:41 +0000 2024,@kompascom Biasa....drama mulyono...pura2 kage...,in,NaN
2,Tue Sep 17 23:55:11 +0000 2024,@fufu__fafa Jamet menjijikkan. Mesum pula.,in,"Jakarta Capital Region, Indone"
3,Tue Sep 17 23:55:00 +0000 2024,@satriohendri Nebeng temen...ato nebeng sopir ...,in,NaN
4,Tue Sep 17 23:54:58 +0000 2024,@andikamalreza kasian org ini.. model penakut....,in,NaN


In [60]:
variations=data['lang'].unique()
print(variations)

['in']


In [78]:
data = data.drop(['lang'], axis=1)

In [79]:
text = data

def extract_mentions(text):
    mentions = re.findall(r'@\w+', text)
    return mentions

text['mentions'] = text['full_text'].apply(extract_mentions)

mentions = text['mentions'].sum()
mention_count = Counter(mentions)
for mention, count in (mention_count.most_common(6)):
    print(f"{mention:<20} | {count:>5} times")


@fufu__fafa          |   142 times
@msaid_didu          |    99 times
@habiburokhman       |    85 times
@prabowo             |    83 times
@JimlyAs             |    50 times
@geloraco            |    48 times


In [80]:
def clean_text(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data['full_text'] = data['full_text'].apply(clean_text)

In [81]:
data['full_text'].head()

0    Yuk ganyang Fufufafa sesuai anjuran HRS PKS pa...
1    Biasadrama mulyonopura kaget fufufafa fufufafaaaa
2                         Jamet menjijikkan Mesum pula
3    Nebeng temenato nebeng sopir temen Fufufafa fu...
4    kasian org ini model penakut Sama fufufafa yg ...
Name: full_text, dtype: object

In [10]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\calys\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\calys\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
#tokeniza
data['tokens'] = data['full_text'].apply(word_tokenize)

#stopwords
stop_words = set(stopwords.words('indonesian'))
data['tokens'] = data['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

#stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

data['stemmed_tokens'] = data['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

#join
data['clean_text'] = data['stemmed_tokens'].apply(lambda x: ' '.join(x))

In [12]:
data.head()

,created_at,full_text,location,mentions,tokens,stemmed_tokens,clean_text
0,Tue Sep 17 23:55:49 +0000 2024,Yuk ganyang Fufufafa sesuai anjuran HRS PKS pa...,NaN,"[@UusRsd, @msaid_didu]","[Yuk, ganyang, Fufufafa, sesuai, anjuran, HRS,...","[yuk, ganyang, fufufafa, sesuai, anjur, hrs, p...",yuk ganyang fufufafa sesuai anjur hrs pks bera...
1,Tue Sep 17 23:55:41 +0000 2024,Biasadrama mulyonopura kaget fufufafa fufufafaaaa,NaN,[@kompascom],"[Biasadrama, mulyonopura, kaget, fufufafa, fuf...","[biasadrama, mulyonopura, kaget, fufufafa, fuf...",biasadrama mulyonopura kaget fufufafa fufufafaaaa
2,Tue Sep 17 23:55:11 +0000 2024,Jamet menjijikkan Mesum pula,"Jakarta Capital Region, Indone",[@fufu__fafa],"[Jamet, menjijikkan, Mesum]","[jamet, jijik, mesum]",jamet jijik mesum
3,Tue Sep 17 23:55:00 +0000 2024,Nebeng temenato nebeng sopir temen Fufufafa fu...,NaN,[@satriohendri],"[Nebeng, temenato, nebeng, sopir, temen, Fufuf...","[nebeng, temenato, nebeng, sopir, temen, fufuf...",nebeng temenato nebeng sopir temen fufufafa fu...
4,Tue Sep 17 23:54:58 +0000 2024,kasian org ini model penakut Sama fufufafa yg ...,NaN,[@andikamalreza],"[kasian, org, model, penakut, fufufafa, yg, ud...","[kasi, org, model, takut, fufufafa, yg, udah, ...",kasi org model takut fufufafa yg udah hina dia...


In [86]:
data.to_csv('tweets-data/cleaned.csv',index=False)